#### Week 2

Code to complete homework for Week 2 - Experiment Tracking

Homework questions are [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/02-experiment-tracking/homework.md)

In [1]:
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso 
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# Experiment tracking
import mlflow

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location='/home/ubuntu/mlops/notebooks/mlruns/1', creation_time=1716820670098, experiment_id='1', last_update_time=1716820670098, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [3]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet", engine='pyarrow')
df2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet", engine='pyarrow')

In [4]:
# Feature engineering
df1['duration'] =df1["tpep_dropoff_datetime"] -  df1["tpep_pickup_datetime"] 
df1["duration"] = df1["duration"].dt.total_seconds().div(60).astype(float)
df1 = df1[(df1["duration"] <= 60)
                 & (df1["duration"] >= 1)]
cat_feat = ["PULocationID", "DOLocationID"]
num_feat = ['trip_distance']
df1[cat_feat] = df1[cat_feat].astype(str)
train_dicts = df1[cat_feat+num_feat].to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df1["duration"].values

In [5]:
# Prepare Feb dataset
df2['duration'] = df2["tpep_dropoff_datetime"] -  df2["tpep_pickup_datetime"] 
df2["duration"] = df2["duration"].dt.total_seconds().div(60).astype(float)

df2 = df2[(df2["duration"] <= 60)
                 & (df2["duration"] >= 1)]

df2[cat_feat] = df2[cat_feat].astype(str)
test_dicts = df2[cat_feat + num_feat].to_dict(orient="records")
X_test = dv.transform(test_dicts)
y_test = df2["duration"].values


### Running Lasso model with MLFlow

#### Linear Regression

In [8]:
with mlflow.start_run():
    mlflow.set_tag("developer", "me")

    mlflow.log_param("train-data-path", "../data/yellow_traipdata_2023_01.parquet")
    mlflow.log_param("valid-data-path", "../data/yellow_traipdata_2023_02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    
    mlflow.log_metric("rmse", rmse)

    
    # Saves model run information
    mlflow.log_artifact(local_path = "models/lin_reg.bin", artifact_path="models_pickle")


FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor'

#### XGBoost 
(and a few more parameters)

Running once (without parameter search)

In [22]:

with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_test, label=y_test)
    mlflow.set_tag("model", "xgboost")
    best_params = {
        'learning_rate': 0.098,
        'max_depth': 28,
        'min_child_weight': 1.04,
        'objective': 'reg:linear',
        'reg_alpha': 0.019,
        'reg_lambda': 0.0098,
        'seed': 42
    }
    mlflow.log_params(best_params)
    booster = xgb.train(
        params=best_params, 
        dtrain=train, 
        num_boost_round=10,
        evals=[(valid, "validation")],
        early_stopping_rounds=3
    )
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_test, y_pred)
    mlflow.log_metric("rmse", rmse)    
    # Saves preprocessing steps
    with open('models/preprocessor.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    # Below lines show the different ways to save models
    mlflow.log_artifact(local_path = "models/preprocessor.bin", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")

# mlflow.end_run()

/home/ubuntu/.cache/pypoetry/virtualenvs/mlops-lf3P47R2-py3.12/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [16:05:22] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.33370
[1]	validation-rmse:8.68976
[2]	validation-rmse:8.12477
[3]	validation-rmse:7.63166
[4]	validation-rmse:7.20287
[5]	validation-rmse:6.83016
[6]	validation-rmse:6.51094
[7]	validation-rmse:6.23418
[8]	validation-rmse:5.99897
[9]	validation-rmse:5.79799


/home/ubuntu/.cache/pypoetry/virtualenvs/mlops-lf3P47R2-py3.12/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [16:05:45] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


### Loading previously recorded model to make predictions

This comes from the artifacts page in MLFlow

In [14]:

logged_model = 'runs:/dc4c27aef42b4958a52aa5b04eaf004c/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(pd.DataFrame(y_test))

/home/ubuntu/.cache/pypoetry/virtualenvs/mlops-lf3P47R2-py3.12/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [15:48:42] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


Running *with* parameter search

In [9]:
def objective(params):
    with mlflow.start_run():
        train = xgb.DMatrix(X_train, label=y_train)
        valid = xgb.DMatrix(X_test, label=y_test)
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params, 
            dtrain=, 
            num_boost_round=10,
            evals=[(valid, "validation")],
            early_stopping_rounds=3
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_test, y_pred)
        mlflow.log_metric("rmse", rmse)    
    # Saves preprocessing steps
    with open('models/preprocessor.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact(local_path = "models/preprocessor.bin", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")

In [10]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    # 'learning_rate': hp.loguniform('learning_rate', -3, 0),
    # 'reg_alpha': hp.loguniform('reg_alpha', -4, -1),
    # 'reg_alpha': hp.loguniform('reg_alpha', -3, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -4, -1),
    # 'min_child_weight': hp.loguniform('min_child_weight', -1, 2),
    'objective': 'reg:linear',
    'seed': 42 
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

job exception: name 'train' is not defined



  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]


NameError: name 'train' is not defined